In [1]:
# To run benchmark script, you will need to install XGBoost 
# (pip install XGBoost)

import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
import warnings
warnings.filterwarnings("ignore")

def load_breast_data():
    breast = load_breast_cancer()
    feature_names = list(breast.feature_names)
    X, y = pd.DataFrame(breast.data, columns=feature_names), breast.target
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X,
            'y': y,
        },
    }
    return dataset


def load_adult_data():
    df = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None)
    df.columns = [
        "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
        "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
        "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
    ]
    train_cols = df.columns[0:-1]
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label]

    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }

    return dataset

def load_heart_data():
    # https://www.kaggle.com/ronitf/heart-disease-uci
    df = pd.read_csv(r'C:\develop\data\heart-disease-uci\heart.csv')
    train_cols = df.columns[0:-1]
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label]
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }
    
    return dataset


def load_credit_data():
    # https://www.kaggle.com/mlg-ulb/creditcardfraud
    df = pd.read_csv(r'C:\develop\data\creditcardfraud\creditcard.csv')
    train_cols = df.columns[0:-1]
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label]
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }
    
    return dataset


def load_telco_churn_data():
    # https://www.kaggle.com/blastchar/telco-customer-churn/downloads/WA_Fn-UseC_-Telco-Customer-Churn.csv/1
    df = pd.read_csv(r'C:\develop\data\telco-customer-churn\WA_Fn-UseC_-Telco-Customer-Churn.csv')
    train_cols = df.columns[1:-1] # First column is an ID
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label] # 'Yes, No'
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }
    
    return dataset

In [2]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.linear_model import SGDClassifier, LogisticRegression

from interpret.glassbox import ExplainableBoostingClassifier, APLRClassifier


def format_n(x):
    return "{0:.3f}".format(x)

def process_model(clf, name, X, y, n_splits=3):
    # Evaluate model
    ss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.25, random_state=1337)
    scores = cross_validate(
        clf, X, y, scoring='roc_auc', cv=ss,
        n_jobs=None, return_estimator=True
    )

    record = dict()
    record['model_name'] = name
    record['fit_time_mean'] = format_n(np.mean(scores['fit_time']))
    record['fit_time_std'] = format_n(np.std(scores['fit_time']))
    record['test_score_mean'] = format_n(np.mean(scores['test_score']))
    record['test_score_std'] = format_n(np.std(scores['test_score']))

    return record



def benchmark_models(dataset_name, X, y, ct=None, n_splits=3, random_state=1337):
    if ct is None:
        is_cat = np.array([dt.kind == 'O' for dt in X.dtypes])
        cat_cols = X.columns.values[is_cat]
        num_cols = X.columns.values[~is_cat]

        cat_ohe_step = ('ohe', OneHotEncoder(sparse=False,
                                             handle_unknown='ignore'))

        cat_pipe = Pipeline([cat_ohe_step])
        num_pipe = Pipeline([('identity', FunctionTransformer())])
        transformers = [
            ('cat', cat_pipe, cat_cols),
            ('num', num_pipe, num_cols)
        ]
        ct = ColumnTransformer(transformers=transformers)

    records = []

    summary_record = {}
    summary_record['dataset_name'] = dataset_name
    print()
    print('-' * 78)
    print(dataset_name)
    print('-' * 78)
    print(summary_record)
    print()

    pipe = Pipeline([
        ('ct', ct),
        ('std', StandardScaler()),
        ('lr', LogisticRegression(random_state=random_state)),
    ])
    record = process_model(pipe, 'lr', X, y, n_splits=n_splits)
    print(record)
    record.update(summary_record)
    records.append(record)

    pipe = Pipeline([
        ('ct', ct),
        # n_estimators updated from 10 to 100 due to sci-kit defaults changing in future versions
        ('rf-100', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=random_state)),
    ])
    record = process_model(pipe, 'rf-100', X, y, n_splits=n_splits)
    print(record)
    record.update(summary_record)
    records.append(record)
    
    pipe = Pipeline([
        ('ct', ct),
        ('xgb', XGBClassifier(random_state=random_state, eval_metric='logloss')),
    ])
    record = process_model(pipe, 'xgb', X, y, n_splits=n_splits)
    print(record)
    record.update(summary_record)
    records.append(record)

    pipe = Pipeline([
        ('ct', ct),
        ('aplr', APLRClassifier(random_state=random_state, num_first_steps_with_linear_effects_only=2000)),
    ])
    record = process_model(pipe, 'aplr', X, y, n_splits=n_splits)
    print(record)
    record.update(summary_record)
    records.append(record)

    # No pipeline needed due to EBM handling string datatypes
    ebm_inter = ExplainableBoostingClassifier(n_jobs=-1, random_state=random_state)
    record = process_model(ebm_inter, 'ebm', X, y, n_splits=n_splits)
    print(record)
    record.update(summary_record)
    records.append(record)

    return records

In [3]:
results = []
n_splits = 3

In [4]:
dataset = load_heart_data()
result = benchmark_models('heart', dataset['full']['X'], dataset['full']['y'], n_splits=n_splits)
results.append(result)


------------------------------------------------------------------------------
heart
------------------------------------------------------------------------------
{'dataset_name': 'heart'}

{'model_name': 'lr', 'fit_time_mean': '0.008', 'fit_time_std': '0.002', 'test_score_mean': '0.895', 'test_score_std': '0.030'}
{'model_name': 'rf-100', 'fit_time_mean': '1.002', 'fit_time_std': '1.119', 'test_score_mean': '0.890', 'test_score_std': '0.008'}
{'model_name': 'xgb', 'fit_time_mean': '0.482', 'fit_time_std': '0.561', 'test_score_mean': '0.851', 'test_score_std': '0.018'}
{'model_name': 'ebm', 'fit_time_mean': '1.495', 'fit_time_std': '0.442', 'test_score_mean': '0.898', 'test_score_std': '0.013'}


In [5]:
dataset = load_breast_data()
result = benchmark_models('breast-cancer', dataset['full']['X'], dataset['full']['y'], n_splits=n_splits)
results.append(result)


------------------------------------------------------------------------------
breast-cancer
------------------------------------------------------------------------------
{'dataset_name': 'breast-cancer'}

{'model_name': 'lr', 'fit_time_mean': '0.006', 'fit_time_std': '0.000', 'test_score_mean': '0.995', 'test_score_std': '0.005'}
{'model_name': 'rf-100', 'fit_time_mean': '0.170', 'fit_time_std': '0.004', 'test_score_mean': '0.992', 'test_score_std': '0.009'}
{'model_name': 'xgb', 'fit_time_mean': '0.084', 'fit_time_std': '0.000', 'test_score_mean': '0.992', 'test_score_std': '0.010'}
{'model_name': 'ebm', 'fit_time_mean': '2.616', 'fit_time_std': '0.802', 'test_score_mean': '0.995', 'test_score_std': '0.006'}


In [6]:
dataset = load_adult_data()
result = benchmark_models('adult', dataset['full']['X'], dataset['full']['y'], n_splits=n_splits)
results.append(result)


------------------------------------------------------------------------------
adult
------------------------------------------------------------------------------
{'dataset_name': 'adult'}

{'model_name': 'lr', 'fit_time_mean': '1.215', 'fit_time_std': '0.032', 'test_score_mean': '0.907', 'test_score_std': '0.003'}
{'model_name': 'rf-100', 'fit_time_mean': '1.599', 'fit_time_std': '0.016', 'test_score_mean': '0.903', 'test_score_std': '0.002'}
{'model_name': 'xgb', 'fit_time_mean': '4.047', 'fit_time_std': '0.052', 'test_score_mean': '0.927', 'test_score_std': '0.001'}
{'model_name': 'ebm', 'fit_time_mean': '26.368', 'fit_time_std': '0.637', 'test_score_mean': '0.928', 'test_score_std': '0.002'}


In [7]:
dataset = load_credit_data()
result = benchmark_models('credit-fraud', dataset['full']['X'], dataset['full']['y'], n_splits=n_splits)
results.append(result)


------------------------------------------------------------------------------
credit-fraud
------------------------------------------------------------------------------
{'dataset_name': 'credit-fraud'}

{'model_name': 'lr', 'fit_time_mean': '3.637', 'fit_time_std': '0.110', 'test_score_mean': '0.979', 'test_score_std': '0.002'}
{'model_name': 'rf-100', 'fit_time_mean': '61.624', 'fit_time_std': '4.390', 'test_score_mean': '0.950', 'test_score_std': '0.007'}
{'model_name': 'xgb', 'fit_time_mean': '46.220', 'fit_time_std': '0.495', 'test_score_mean': '0.981', 'test_score_std': '0.003'}
{'model_name': 'ebm', 'fit_time_mean': '58.282', 'fit_time_std': '1.429', 'test_score_mean': '0.981', 'test_score_std': '0.003'}


In [8]:
dataset = load_telco_churn_data()
result = benchmark_models('telco-churn', dataset['full']['X'], dataset['full']['y'], n_splits=3)
results.append(result)


------------------------------------------------------------------------------
telco-churn
------------------------------------------------------------------------------
{'dataset_name': 'telco-churn'}

{'model_name': 'lr', 'fit_time_mean': '0.095', 'fit_time_std': '0.009', 'test_score_mean': '0.849', 'test_score_std': '0.005'}
{'model_name': 'rf-100', 'fit_time_mean': '0.406', 'fit_time_std': '0.023', 'test_score_mean': '0.824', 'test_score_std': '0.004'}
{'model_name': 'xgb', 'fit_time_mean': '0.851', 'fit_time_std': '0.044', 'test_score_mean': '0.828', 'test_score_std': '0.010'}
{'model_name': 'ebm', 'fit_time_mean': '3.675', 'fit_time_std': '0.899', 'test_score_mean': '0.852', 'test_score_std': '0.006'}


In [9]:
records = [item for result in results for item in result]
record_df = pd.DataFrame.from_records(records)[['dataset_name', 'model_name', 'test_score_mean', 'test_score_std']]
record_df.to_csv('ebm-perf-classification-overnight.csv')